## <center> Attacks through Environment Variables

### What are environment variables?

- Consist of names that have values assigned to them. 
- Provide a way to influence the behaviour of software on the system.
- For example:
  - The PATH environment variable provides a semicolon-separated list of directories where executable programs are stored

### Working principles of environment variables

- Process locality: The values of EV are local. If two terminals/shells are opened, and we change a system EV in one of the shell, the other will not be impacted. 
- Inheritance: When a parent process creates a child process, the child process inherits all the EVs the parent process had.
- Case sensititivy: The names of EVs are case sensitive.

#### Exercise
- Open a new terminal
- Run 'printenv' to view all system EVs inside the SEED VM

### How do C programs access environment variables?

- Third argument provided to the `main()` function: `envp[]`, or
- Global variable `environ`

In [ ]:
%%writefile source/print_env.c
#include <stdio.h>
void main(int argc, char* argv[], char* envp[])
{
   int i = 0;
   while (envp[i] !=NULL) {
      printf("%s\n", envp[i++]);
   }
}

#### Exercise

- Compile and run the `print_env` program (see previous execution example from the set_uid lecture)

In [ ]:
%%writefile source/print_env_environ.c
#include <stdio.h>

extern char** environ;
void main(int argc, char* argv[], char* envp[])
{
   int i = 0;
   while (environ[i] != NULL) {
      printf("%s\n", environ[i++]);
   }
}

#### Exercise

- Compile and run the `print_env_environ` program (see previous execution example from the set_uid lecture)
- Is the printed list of EVs any different than the one printed by `print_env`?

### How the EVs are passed to a process?

- If this is a new process (created using `fork()`)
  - EV list is a duplicate of parent process
- If this is a new program inside an existing process (created using `execve()`)
  - EV list is overwritten with the new program's
  - Need to pass an EV list (existing EVs or new ones) to `execve()`
  - https://git.kernel.org/pub/scm/linux/kernel/git/torvalds/linux.git/tree/fs/exec.c#l1376

In [ ]:
%%writefile source/run_execve.c
#include <stdio.h>

extern char ** environ;
void main(int argc, char* argv[], char* envp[])
{
  int i = 0; char* v[2]; char* newenv[3];
  if (argc < 2) return;

  // Construct the argument array
  v[0] = "/usr/bin/env";   v[1] = NULL;

  // Construct the environment variable array
  newenv[0] = "AAA=aaa"; newenv[1] = "BBB=bbb"; newenv[2] = NULL;

  switch(argv[1][0]) {
    case '1': // Passing no environment variable.
       execve(v[0], v, NULL);
    case '2': // Passing a new set of environment variables.
       execve(v[0], v, newenv);
    case '3': // Passing all the environment variables.
       execve(v[0], v, environ);
    default:
       execve(v[0], v, NULL);
  }
}

#### Exercise

- Save and compile `run_execve`
- Execute `run_execve` three times with the three input options

### Shell variables versus environment variables

- Are not the same thing
- Different but related concepts

- Recall from Introduction to Linux: Shell is a command line interface  for users to interact with an operating system (Linux)
- Shell variables are internal variables maintained by a shell program
- User can create, assign, and delete shell variables
- Environment variables can become shell variables and vice versa

When a shell program starts (a terminal is opened)
- The shell defines a shell variable for each environment variable of the process (the shell process)
- Same name, same value
- The shell can easily get the value of the environment variables by referring to its own shell variables

#### Exercise

```
$ cat /proc/$$environ
$ strings /proc/$$/environ | grep LOGNAME
$ echo $LOGNAME
$ LOGNAME=ram
$ echo $LOGNAME
$ strings /proc/$$/environ | grep LOGNAME
```

Shell variables affect the environment variable of child processes
- Technically, each (any) program is executed from inside a shell (from terminal)
- Two types of shell variables will be provided to the new program

<center> <img src="figure/environment-variables/ev2.png" width="400"/>

<center> <img src="figure/environment-variables/ev1.png" width="400"/>

### Attack surfaces caused by environment variables

<center> <img src="figure/environment-variables/ev3.png" width="400"/>

<center> <img src="figure/environment-variables/ev4.png" width="400"/>

<center> <img src="figure/environment-variables/ev5.png" width="400"/>